In [1]:
import faiss
from faiss import IndexIVFFlat, IndexFlatL2
import numpy as np

f = open("../build/embeddings.bin", "rb")
num_embeddings = np.fromfile(f, dtype=np.int32, count=1)[0]
d = np.fromfile(f, dtype=np.int32, count=1)[0]
d = int(d)
print(f"num_embeddings: {num_embeddings}, dimension: {d}")
# num_embeddings = 1000000
embeddings = np.fromfile(f, dtype=np.float32, count=num_embeddings * d)
embeddings = embeddings.reshape(num_embeddings, d)
f.close()
print(f"embeddings shape: {embeddings.shape}")

num_embeddings: 266205513, dimension: 128
embeddings shape: (266205513, 128)


In [2]:
# randomly sample 2097152 embeddings from the dataset
sample_size = 2097152
sample = embeddings[np.random.choice(num_embeddings, sample_size, replace=False)]
hnsw = faiss.IndexHNSWFlat(d, 16, faiss.METRIC_INNER_PRODUCT)
hnsw.hnsw.efConstruction = 500
hnsw.add(sample)

In [3]:
hnsw.hnsw.efSearch = 200
ivf = faiss.IndexIVFFlat(hnsw, d, sample_size, faiss.METRIC_INNER_PRODUCT)
ivf.make_direct_map(True)
ivf.add(embeddings)

In [ ]:
faiss.write_index(ivf, "../build/ivf_hnsw_2097152.faiss")

: 